In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import pickle
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
X_data = []
y_label = []
with open('../input/khaiphadulieu/topic_detection_train.v1.0.txt', 'r', encoding="utf-8") as f:
    for lines in f.readlines():
        index = lines.find(' ')
        y_label.append(lines[:index])
        lines = lines[index:]
        lines = gensim.utils.simple_preprocess(lines)
#         lines = re.sub(r"[^\w\d\s]"," ",lines,flags=re.UNICODE)
#         lines = re.sub("[0-9]"," ",lines,flags=re.UNICODE)
#         lines = lines.split()
        lines = ' '.join(lines)
        lines = ViTokenizer.tokenize(lines)
        X_data.append(lines)
pickle.dump(X_data, open('X_data.pkl', 'wb'))
pickle.dump(y_label, open('y_label.pkl', 'wb'))

In [ ]:
import pickle
X_data = pickle.load(open('X_data.pkl', 'rb'))
y_label = pickle.load(open('y_label.pkl', 'rb'))
print(X_data[0])

In [ ]:
# # tạo stopword bằng cách đếm số từ phổ biến
# uniqueWords = []
# numOfWords = []
# for bagOfWords in X_data:
#     uniqueWords = set(uniqueWords).union(set(bagOfWords.split(' ')))
# numOfWords = dict.fromkeys(uniqueWords, 0)
# for bagOfWords in X_data:
#     for word in set(bagOfWords.split(' ')).union():
#         numOfWords[word] += bagOfWords.count(word)
# numOfWords_sorted = sorted(numOfWords.items(), key=lambda item: item[1])
# i = 0
# word_stop = []
# for word in numOfWords_sorted:
#     print(numOfWords_sorted[len(numOfWords)-1-i])
#     i+=1
#     w,num = numOfWords_sorted[len(numOfWords)-1-i]
#     if w == "nước":
#         break
#     word_stop.append(w)

In [ ]:
# #tạo stopword bằng cách liệt kê
# word_stop = "ng bị bởi cả các cái cần càng chỉ chiếc cho chứ chưa chuyện có có_thể cứ của cùng cũng đã đang đây để đến_nỗi đều điều do đó được dưới gì khi không là lại lên lúc mà mỗi một_cách này nên nếu ngay nhiều như nhưng những nơi nữa phải qua ra rằng rằng rất rất rồi sau sẽ so sự tại theo thì trên trước từ từng và vẫn vào vậy vì việc với vừa"
# word_stop = word_stop.split()
# pickle.dump(word_stop, open('word_stop.pkl', 'wb'))
# word_stops = pickle.load(open('word_stop.pkl', 'rb'))
# for index in range(len(X_data)):
#     for word_stop in word_stops:
#         if word_stop in X_data[index]:
#             X_data[index] = X_data[index].replace(word_stop+' ', '')
# pickle.dump(X_data, open('X_data.pkl', 'wb'))
# X_data = pickle.load(open('X_data.pkl', 'rb'))

In [ ]:
#require
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=42)

In [ ]:
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(X_train)
X_data_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_train)
# assume that we don't have test set before
X_test_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_test)
from sklearn.decomposition import TruncatedSVD
svd_ngram_char = TruncatedSVD(n_components=300, random_state=42)
svd_ngram_char.fit(X_data_tfidf_ngram_char)

X_data_tfidf_ngram_char_svd = svd_ngram_char.transform(X_data_tfidf_ngram_char)
# assume that we don't have test set before
X_test_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_tfidf_ngram_char)
pickle.dump(X_data_tfidf_ngram_char_svd, open('X_data_tfidf_ngram_char_svd.pkl', 'wb'))
pickle.dump(X_test_tfidf_ngram_char_svd, open('X_test_tfidf_ngram_char_svd.pkl', 'wb'))

In [ ]:
#require
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_train)
y_test_n = encoder.fit_transform(y_test)
y_data_encoder = encoder.fit_transform(y_label)
encoder.classes_

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_train) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_train)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)


In [ ]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X_train)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

In [ ]:
#require
X_data_tfidf_ngram_char_svd = pickle.load(open('X_data_tfidf_ngram_char_svd.pkl', 'rb'))
X_test_tfidf_ngram_char_svd = pickle.load(open('X_test_tfidf_ngram_char_svd.pkl', 'rb'))
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn import neighbors
import matplotlib.pyplot as plt
clf = neighbors.KNeighborsClassifier(n_neighbors = 5, p = 2, weights = 'distance')
clf.fit(X_data_tfidf_ngram_char_svd, y_data_n)
y_pred = clf.predict(X_test_tfidf_ngram_char_svd)
print("Accuracy of KNN: %.2f %%" %(100*accuracy_score(y_test_n, y_pred)))
# Save model
pickle.dump(clf, open('neighbors.pkl', 'wb'))

In [ ]:
from sklearn.linear_model import LogisticRegression
clf_logic = LogisticRegression(random_state=0).fit(X_data_tfidf_ngram_char_svd, y_data_n)
y_pred_logic = clf_logic.predict(X_test_tfidf_ngram_char_svd)
print("Accuracy of logistic: %.2f %%" %(100*accuracy_score(y_test_n, y_pred_logic)))
pickle.dump(clf_logic, open('LogisticRegression.pkl', 'wb'))

In [ ]:
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
# kernel = DotProduct() + WhiteKernel()
# gpr = GaussianProcessRegressor(kernel=kernel,random_state=0).fit(X_data_tfidf_ngram_char_svd, y_data_n)
# y_pred_gpr, sigma = gpr.predict(X_test_tfidf_ngram_char_svd, return_std=True)
# # print("Accuracy of KNN: %.2f %%" %(100*accuracy_score(y_test_n, y_pred_gpr)))

In [ ]:
# from sklearn.decomposition import TruncatedSVD
# from sklearn.svm import SVC
# for n in range(100,3000,100):
#     svd_ngram_char = TruncatedSVD(n_components=n, random_state=42)
#     svd_ngram_char.fit(X_data_tfidf_ngram_char)

#     X_data_tfidf_ngram_char_svd = svd_ngram_char.transform(X_data_tfidf_ngram_char)
#     # assume that we don't have test set before
#     X_test_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_tfidf_ngram_char)
#     # C_2d_range = [1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9]
#     # gamma_2d_range = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2]
#     # for C in C_2d_range:
#     #     for gamma in gamma_2d_range:
#     #         svm = SVC(kernel='rbf', random_state=1, C=C, gamma=gamma)
#     #         svm.fit(X_data_tfidf_ngram_char_svd, y_data_n)
#     #         y_pred_svm = svm.predict(X_test_tfidf_ngram_char_svd)
#     #         print("c = %.9f , gamma = %.9f" %(C,gamma))
#     #         print("Accuracy of SVC: %.2f %%" %(100*accuracy_score(y_test_n, y_pred_svm)))
#     svm = SVC(kernel='rbf', random_state=1, gamma=0.1, C=10)
#     svm.fit(X_data_tfidf_ngram_char_svd, y_data_n)
#     y_pred_svm = svm.predict(X_test_tfidf_ngram_char_svd)
#     print("n = %d" %(n))
#     print("Accuracy of KNN: %.2f %%" %(100*accuracy_score(y_test_n, y_pred_svm)))

In [ ]:
from sklearn.svm import SVC
# C_2d_range = [1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9]
# gamma_2d_range = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2]
# for C in C_2d_range:
#     for gamma in gamma_2d_range:
#         svm = SVC(kernel='rbf', random_state=1, C=C, gamma=gamma)
#         svm.fit(X_data_tfidf_ngram_char_svd, y_data_n)
#         y_pred_svm = svm.predict(X_test_tfidf_ngram_char_svd)
#         print("c = %.9f , gamma = %.9f" %(C,gamma))
#         print("Accuracy of SVC: %.2f %%" %(100*accuracy_score(y_test_n, y_pred_svm)))
svm = SVC(kernel='rbf', random_state=1, gamma=0.1, C=10)
svm.fit(X_data_tfidf_ngram_char_svd, y_data_n)
y_pred_svm = svm.predict(X_test_tfidf_ngram_char_svd)
print("Accuracy of KNN: %.2f %%" %(100*accuracy_score(y_test_n, y_pred_svm)))
# C = 0.1, gamma = 1 => 82.41%
# C = 1, gamma = 1 => 87.66%
# C = 10, gamma = 0.1 => 87.91%
# c = 100.000000000 , gamma = 0.010000000 Accuracy of SVC: 87.88 %
pickle.dump(svm, open('svm.pkl', 'wb'))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
regressor = RandomForestClassifier(n_estimators=1000, random_state=0)
regressor.fit(X_data_tfidf_ngram_char_svd, y_data_n)
y_pred_forest = regressor.predict(X_test_tfidf_ngram_char_svd)
print("Accuracy of Randomforest: %.2f %%" %(100*accuracy_score(y_test_n, y_pred_forest)))
pickle.dump(regressor, open('RandomForestClassifier.pkl', 'wb'))

In [ ]:
X_test_real=[]
root_test_data = []
with open('../input/khaiphadulieutest/topic_detection_test_unlabel.v1.0.txt', 'r', encoding="utf-8") as f:
    for lines in f.readlines():
        root_test_data.append(lines)
        lines = gensim.utils.simple_preprocess(lines)
#         lines = re.sub(r"[^\w\d\s]"," ",lines,flags=re.UNICODE)
#         lines = re.sub("[0-9]"," ",lines,flags=re.UNICODE)
#         lines = lines.split()
        lines = ' '.join(lines)
        lines = ViTokenizer.tokenize(lines)
        X_test_real.append(lines)
pickle.dump(X_test_real, open('X_test_real.pkl', 'wb'))
X_test_real_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_test_real)
X_test_real_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_real_tfidf_ngram_char)

In [ ]:
# SVM
svm = pickle.load(open("svm.pkl", 'rb'))
y_pred = svm.predict(X_test_real_tfidf_ngram_char_svd)
print(y_pred[0])

In [ ]:
with open("nhom1_TD_SOL2.txt", mode="a", encoding="utf-8") as f:
    for index in range(len(y_pred)):
        f.write(encoder.classes_[y_pred[index]]+' '+root_test_data[index])

In [ ]:
# neighbors
neighbors = pickle.load(open("neighbors.pkl", 'rb'))
y_pred_neighbors = neighbors.predict(X_test_real_tfidf_ngram_char_svd)
print(y_pred_neighbors[0])
with open("nhom1_TD_SOL3.txt", mode="a", encoding="utf-8") as f:
    for index in range(len(y_pred_neighbors)):
        f.write(encoder.classes_[y_pred_neighbors[index]]+' '+root_test_data[index])

In [ ]:
# LogisticRegression
logis = pickle.load(open("LogisticRegression.pkl", 'rb'))
y_pred_logis = logis.predict(X_test_real_tfidf_ngram_char_svd)
print(y_pred_logis[0])
with open("nhom1_TD_SOL4.txt", mode="a", encoding="utf-8") as f:
    for index in range(len(y_pred_logis)):
        f.write(encoder.classes_[y_pred_logis[index]]+' '+root_test_data[index])

In [ ]:
# LogisticRegression
forest = pickle.load(open("RandomForestClassifier.pkl", 'rb'))
y_pred_forest = forest.predict(X_test_real_tfidf_ngram_char_svd)
print(y_pred_forest[0])
with open("nhom1_TD_SOL5.txt", mode="a", encoding="utf-8") as f:
    for index in range(len(y_pred_forest)):
        f.write(encoder.classes_[y_pred_forest[index]]+' '+root_test_data[index])